In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
os.environ["KERAS_BACKEND"]="jax" 
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

In [8]:
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0,preprocess_input
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,CSVLogger

In [3]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar100.load_data()

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step


In [4]:
x_train=tf.image.resize(x_train,(64,64)).numpy()
x_test=tf.image.resize(x_test,(64,64)).numpy()

I0000 00:00:1744612984.410549      31 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1744612984.413114      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [5]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,random_state=42,test_size=0.25,stratify=y_train)

In [6]:
datagen=ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

In [9]:
base_model=EfficientNetV2B0(
    weights='imagenet',
    include_top=False, 
    input_shape=(64,64,3) 
)

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [10]:
base_model.trainable=False

In [11]:
model=models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(512,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(256,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(100,activation='softmax') 
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetv2-b0 (Functional)       │ (None, 2, 2, 1280)          │       5,919,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       1,311,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 100)                 │          25,700 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,920,052 (30.21 MB)

 Trainable params: 1,997,156 (7.62 MB)

 Non-trainable params: 5,922,896 (22.59 MB)

In [12]:
early_stopper=EarlyStopping(restore_best_weights=True,monitor="val_loss",patience=5)

In [13]:
checkpoint=ModelCheckpoint('best_model.keras',monitor='val_accuracy',save_best_only=True)

In [14]:
csv_logger=CSVLogger('training_log.csv',append=True)

In [15]:
import time
start_time=time.time()

In [16]:
model.compile(optimizer=Adam(learning_rate=1e-4),loss='sparse_categorical_crossentropy',metrics=["accuracy"],jit_compile=True)

In [17]:
history=model.fit(datagen.flow(x_train,y_train),epochs=20,batch_size=32,callbacks=[early_stopper],validation_data=[x_val,y_val],verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 90s 69ms/step - accuracy: 0.0513 - loss: 4.8366 - val_accuracy: 0.3574 - val_loss: 2.6883
Epoch 2/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 63s 49ms/step - accuracy: 0.2134 - loss: 3.3801 - val_accuracy: 0.4316 - val_loss: 2.2501
Epoch 3/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 58s 49ms/step - accuracy: 0.2753 - loss: 2.9600 - val_accuracy: 0.4643 - val_loss: 2.0716
Epoch 4/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 57s 49ms/step - accuracy: 0.3093 - loss: 2.7576 - val_accuracy: 0.4819 - val_loss: 1.9659
Epoch 5/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 57s 49ms/step - accuracy: 0.3419 - loss: 2.6076 - val_accuracy: 0.4945 - val_loss: 1.9030
Epoch 6/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 57s 49ms/step - accuracy: 0.3628 - loss: 2.5107 - val_accuracy: 0.5064 - val_loss: 1.8468
Epoch 7/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 57s 49ms/step - accuracy: 0.3689 - loss: 2.4620 - val_accuracy: 0.5139 - val_loss: 1.8117
Epoch 8/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 56s 48ms/step - accuracy: 0.3814 -

In [18]:
head_training_time=time.time()-start_time

In [19]:
base_model.trainable=True

In [20]:
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],jit_compile=True)

In [21]:
start_fine_tune=time.time()

In [22]:
fine_tune_history=model.fit(
    datagen.flow(x_train,y_train,batch_size=32),
    validation_data=(x_val,y_val),
    epochs=10,
    callbacks=[early_stopper,csv_logger],
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


E0414 07:04:51.255189      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 07:04:51.438532      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 07:04:51.868575      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 07:04:52.085100      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 07:04:52.291966      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 07:04:52.482742      31 gpu_timer.cc:82] Del

 519/1172 ━━━━━━━━━━━━━━━━━━━━ 1:01 95ms/step - accuracy: 0.1997 - loss: 3.6384

E0414 07:06:06.395703      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 07:06:06.579952      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 07:06:07.010983      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 07:06:07.227385      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 07:06:07.435287      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 07:06:07.628563      31 gpu_timer.cc:82] Del

1172/1172 ━━━━━━━━━━━━━━━━━━━━ 155s 109ms/step - accuracy: 0.2127 - loss: 3.5289 - val_accuracy: 0.3782 - val_loss: 2.4778
Epoch 2/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 90s 62ms/step - accuracy: 0.2980 - loss: 2.9238 - val_accuracy: 0.4504 - val_loss: 2.1132
Epoch 3/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 72s 61ms/step - accuracy: 0.3483 - loss: 2.6273 - val_accuracy: 0.4889 - val_loss: 1.9186
Epoch 4/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 71s 60ms/step - accuracy: 0.3843 - loss: 2.4002 - val_accuracy: 0.5158 - val_loss: 1.7886
Epoch 5/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 72s 61ms/step - accuracy: 0.4042 - loss: 2.2770 - val_accuracy: 0.5437 - val_loss: 1.6888


In [23]:
fine_tune_time=time.time()-start_fine_tune

In [24]:
test_loss,test_accuracy=model.evaluate(x_test,y_test,verbose=1)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")
print(f"Total training time: {head_training_time + fine_tune_time:.2f} seconds")


Test Accuracy: 38.92%
Total training time: 1644.28 seconds


In [25]:
val_loss,val_accuracy=model.evaluate(x_val,y_val)
print(f"Validation accuracy: {val_accuracy * 100:.4f}%")

391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.3792 - loss: 2.4893
Validation accuracy: 37.8240%


In [26]:
import numpy as np
y_pred=model.predict(x_test)
y_pred_class=np.argmax(y_pred,axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step


In [27]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,classification_report


accuracy=accuracy_score(y_test,y_pred_class)
precision=precision_score(y_test,y_pred_class,average='weighted')
recall=recall_score(y_test,y_pred_class,average='weighted')
f1=f1_score(y_test,y_pred_class,average='weighted')


print("\nPerformance Metrics Breakdown for EfficientNetV2B0:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Performance Metrics Breakdown for EfficientNetV2B0:
Accuracy: 0.3892
Precision: 0.3889
Recall: 0.3892
F1-Score: 0.3832


In [28]:
import pandas as pd
results=pd.DataFrame({
    "Model": ["EfficientNetV2B0"],
    "Test Accuracy": [test_accuracy],
    "Validation Accuracy":[val_accuracy],
    "Test Loss": [test_loss],
    "Head Training Time (s)": [head_training_time],
    "Fine-tuning Time (s)": [fine_tune_time],
    "Total Params": [model.count_params()]
})

In [29]:
print(results)

              Model  Test Accuracy  Validation Accuracy  Test Loss  \
0  EfficientNetV2B0         0.3892              0.37824   2.453635   

   Head Training Time (s)  Fine-tuning Time (s)  Total Params  
0             1183.816083            460.462514       7920052  


In [30]:
results.to_csv("EfficientNetV2B0.csv",index=False)
print("\nResults saved to EfficientNetV2B0.csv")


Results saved to EfficientNetV2B0.csv
